In [9]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
import chromadb
# from chromadb.utils.embedding_functions import OllamaEmbeddingFunction
from langchain_chroma import Chroma

In [10]:
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
import pandas as pd

books = pd.read_csv('usable_books.csv')

In [12]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missin_description,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,0,Gilead,9780002005883: A NOVEL THAT READERS and critic...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,0,Spider's Web: A Novel,9780002261982: A new 'Christie for Christmas' ...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,0,Rage of angels,"9780006178736: A memorable, mesmerizing heroin..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,0,The Four Loves,9780006280897: Lewis' work on the nature of lo...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,0,The Problem of Pain,"9780006280934: ""In The Problem of Pain, C.S. L..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,0,Mistaken Identity,9788172235222: On A Train Journey Home To Nort...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,0,Journey to the East,9788173031014: This book tells the tale of a m...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623: Wisdom to Create a Life of Pass...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535: This collection of the timeless...


In [13]:
books["tagged_description"]

0       9780002005883: A NOVEL THAT READERS and critic...
1       9780002261982: A new 'Christie for Christmas' ...
2       9780006178736: A memorable, mesmerizing heroin...
3       9780006280897: Lewis' work on the nature of lo...
4       9780006280934: "In The Problem of Pain, C.S. L...
                              ...                        
5192    9788172235222: On A Train Journey Home To Nort...
5193    9788173031014: This book tells the tale of a m...
5194    9788179921623: Wisdom to Create a Life of Pass...
5195    9788185300535: This collection of the timeless...
5196    9789027712059: Since the three volume edition ...
Name: tagged_description, Length: 5197, dtype: object

In [14]:
books["tagged_description"].to_csv("tagged_description.txt", sep="\n", index=False, header=False)

In [ ]:
raw_documents = TextLoader("tagged_description.txt", encoding="utf8").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

In [16]:
# import onnxruntime
# from chromadb.utils.embedding_functions import ONNXMiniLM_L6_V2

# print(onnxruntime.get_available_providers())
db_path = "BookRecommenderWithNomicEmbedTextDB"
client = chromadb.PersistentClient(path=db_path)
# ef = ONNXMiniLM_L6_V2(preferred_providers=['CUDAExecutionProvider'])

# ef = OllamaEmbeddingFunction(
#     model_name="deepseek-r1:7b",
#     url="http://localhost:11434/api/embed",
# )

# db_books_7b = ef(documents)

# Before creating vector database, run this for the first time
db_books_7b = Chroma.from_documents(documents, embedding=OllamaEmbeddings(model="nomic-embed-text:latest"), client=client, persist_directory=db_path)

# After creating vector database, run this
# db_books_7b = Chroma(persist_directory=db_path, embedding_function=OllamaEmbeddings(model="nomic-embed-text:latest"))


In [17]:
search_query = "the ugly aftermath of the conflicts"
search_results = db_books_7b.similarity_search(search_query, k=10)
search_results

[Document(id='70351390-da89-47aa-9236-9966722a8194', metadata={'source': 'tagged_description.txt'}, page_content='9781590172186: Although war was never formally declared, the Algerian War lasted from 1954 to 1962. It caused six French governments to fall, led to the collapse of the Fourth Republic, brought De Gaulle back to power, and came close to provoking a civil war on French soil. More than a million Muslim Algerians died in the conflict and as many European settlers were driven into exile. Above all, the war was marked by an unholy marriage of revolutionary terror and state torture. The war made headlines around the world, and at the time it seemed like a French affair: Now, this brutal and intractable conflict looks less like the last colonial war than the first postmodern one--a full-dress rehearsal for the sort of amorphous struggle that convulsed the Balkans in the 1990s and that is now ravaging Iraq, and in which religion, nationalism, imperialism, and terrorism assume previ

In [18]:
books[books["isbn13"] == int(search_results[0].page_content.split(':')[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missin_description,title_and_subtitle,tagged_description
4836,9781590172186,1590172183,A Savage War of Peace,Alistair Horne,History,http://books.google.com/books/content?id=wuoDD...,"Although war was never formally declared, the ...",2006.0,4.22,608.0,1811.0,0,"A Savage War of Peace: Algeria, 1954-1962",9781590172186: Although war was never formally...


In [19]:
def retrieve_semantic_recommendations(search_query : str, top_k : int = 10) -> pd.DataFrame:
    recs = db_books_7b.similarity_search(search_query, k=50)
    books_list = []
    
    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split(':')[0])]
        
    return books[books["isbn13"].isin(books_list)].head(top_k)

In [20]:
retrieve_semantic_recommendations('the beauty of nature')

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missin_description,title_and_subtitle,tagged_description
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,0,The Four Loves,9780006280897: Lewis' work on the nature of lo...
59,9780007151240,0007151241,The Family Way,Tony Parsons,Parenthood,http://books.google.com/books/content?id=dJEIx...,It should be the most natural thing in the wor...,2005.0,3.51,400.0,2095.0,0,The Family Way,9780007151240: It should be the most natural t...
320,9780060957261,0060957263,Beauty,John O'Donohue,"Body, Mind & Spirit",http://books.google.com/books/content?id=jXgjv...,"Beauty does not linger, it only visits. Yet be...",2005.0,4.45,261.0,622.0,0,Beauty: The Invisible Embrace,"9780060957261: Beauty does not linger, it only..."
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,0,Prodigal Summer: A Novel,9780060959036: Barbara Kingsolver's fifth nove...
442,9780067575208,006757520X,The Sense of Wonder,Rachel Carson,Nature,http://books.google.com/books/content?id=Zee5S...,"First published more than three decades ago, t...",1998.0,4.39,112.0,1160.0,0,The Sense of Wonder,9780067575208: First published more than three...
1642,9780374522599,0374522596,The Control of Nature,John McPhee,Nature,http://books.google.com/books/content?id=p1qKQ...,The Control of Nature is John McPhee's bestsel...,1990.0,4.24,288.0,3365.0,0,The Control of Nature,9780374522599: The Control of Nature is John M...
2806,9780571090242,0571090249,Death of a Naturalist,Seamus Heaney,Poetry,http://books.google.com/books/content?id=et6dA...,Death of a Naturalist marks the auspicious out...,1969.0,4.29,57.0,63.0,0,Death of a Naturalist: Volume one,9780571090242: Death of a Naturalist marks the...
2831,9780571212927,0571212921,A Beautiful Mind,Sylvia Nasar,Mathematicians,http://books.google.com/books/content?id=5ePBQ...,A Beautiful Mind is Sylvia Nasar's award-winni...,2001.0,4.13,461.0,111633.0,0,A Beautiful Mind,9780571212927: A Beautiful Mind is Sylvia Nasa...
3933,9780811205467,0811205460,Turtle Island,Gary Snyder,Poetry,http://books.google.com/books/content?id=xilff...,These Pulitzer Prize-winning poems and essays ...,1974.0,4.09,112.0,3717.0,0,Turtle Island,9780811205467: These Pulitzer Prize-winning po...


In [21]:
books["categories"].value_counts().reset_index()

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
...,...,...
474,Conspiracies,1
475,Brothers and sisters,1
476,Rock musicians,1
477,Community life,1


In [22]:
books["categories"].value_counts().reset_index().query("count > 50")

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
5,Philosophy,117
6,Religion,117
7,Comics & Graphic Novels,116
8,Drama,86
9,Juvenile Nonfiction,57


In [23]:
# books[books["categories"] == "Juvenile Fiction"]

# For Text Classification

category_mapping = {
    'Fiction' : "Fiction",
    'Juvenile Fiction' : "Children's Fiction",
    'Biography & Autobiography' : "Nonfiction",
    'History' : "Nonfiction",
    'Literary Criticism' : "Nonfiction",
    'Philosophy' : "Nonfiction",
    'Religion' : "Nonfiction",
    'Comics & Graphic Novels' : "Fiction",
    'Drama': "Fiction",
    'Juvenile Nonfiction' : "Children's Nonfiction",
    'Science' : "Nonfiction",
    'Poetry' : "Fiction",
}

In [24]:
books["simple_categories"] = books["categories"].map(category_mapping)

In [25]:
books[~books["simple_categories"].isnull()]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missin_description,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,0,Gilead,9780002005883: A NOVEL THAT READERS and critic...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,0,Rage of angels,"9780006178736: A memorable, mesmerizing heroin...",Fiction
8,9780006482079,0006482074,Warhost of Vastmark,Janny Wurts,Fiction,http://books.google.com/books/content?id=uOL0f...,"Tricked once more by his wily half-brother, Ly...",1995.0,4.03,522.0,2966.0,0,Warhost of Vastmark,9780006482079: Tricked once more by his wily h...,Fiction
30,9780006646006,000664600X,Ocean Star Express,Mark Haddon;Peter Sutton,Juvenile Fiction,http://books.google.com/books/content?id=I2QZA...,Joe and his parents are enjoying a summer holi...,2002.0,3.50,32.0,1.0,0,Ocean Star Express,9780006646006: Joe and his parents are enjoyin...,Children's Fiction
46,9780007121014,0007121016,Taken at the Flood,Agatha Christie,Fiction,http://books.google.com/books/content?id=3gWlx...,A Few Weeks After Marrying An Attractive Young...,2002.0,3.71,352.0,8852.0,0,Taken at the Flood,9780007121014: A Few Weeks After Marrying An A...,Fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5178,9781933648279,1933648279,Night Has a Thousand Eyes,Cornell Woolrich,Fiction,http://books.google.com/books/content?id=3Gk6s...,"""Cornell Woolrich's novels define the essence ...",2007.0,3.77,344.0,680.0,0,Night Has a Thousand Eyes,"9781933648279: ""Cornell Woolrich's novels defi...",Fiction
5188,9784770028969,4770028962,Coin Locker Babies,村上龍,Fiction,http://books.google.com/books/content?id=87DJw...,Rescued from the lockers in which they were le...,2002.0,3.75,393.0,5560.0,0,Coin Locker Babies,9784770028969: Rescued from the lockers in whi...,Fiction
5189,9788122200850,8122200850,"Cry, the Peacock",Anita Desai,Fiction,http://books.google.com/books/content?id=_QKwV...,This book is the story of a young girl obsesse...,1980.0,3.22,218.0,134.0,0,"Cry, the Peacock",9788122200850: This book is the story of a you...,Fiction
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535: This collection of the timeless...,Nonfiction


In [26]:
# source: https://huggingface.co/facebook/bart-large-mnli?library=transformers
# Use a pipeline as a high-level helper
from transformers import pipeline

fiction_categories = ['Fiction', 'Nonfiction']

pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForSequenceClassification: ['model.decoder.version', 'model.encoder.version']
- This IS expected if you are initializing TFBartForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForSequenceClassification for predictions without further training.
Device set to use 0


In [27]:
sequence = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop=True)[0]

sequence

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [28]:
pipe(sequence, fiction_categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [29]:
import numpy as np

max_index = np.argmax(pipe(sequence, fiction_categories)["scores"])
max_label = pipe(sequence, fiction_categories)["labels"][max_index]
max_label

'Fiction'

In [30]:
def generate_predictions(sequence, categories):
    predictions = pipe(sequence, categories)
    max_index = np.argmax(predictions["scores"])
    return predictions["labels"][max_index]

In [31]:
from tqdm import tqdm

actual_categories = []
predicted_categories = []

for i in tqdm(range(0, 300)):
    sequence = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop=True)[i]
    predicted_categories += [generate_predictions(sequence, fiction_categories)]
    actual_categories += ["Fiction"]

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [13:12<00:00,  2.64s/it]


In [32]:
for i in tqdm(range(0, 300)):
    sequence = books.loc[books["simple_categories"] == "Nonfiction", "description"].reset_index(drop=True)[i]
    predicted_categories += [generate_predictions(sequence, fiction_categories)]
    actual_categories += ["Nonfiction"]

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [13:18<00:00,  2.66s/it]


In [33]:
predictions_df = pd.DataFrame({"actual_categories": actual_categories, "predicted_categories": predicted_categories})
predictions_df

,actual_categories,predicted_categories
0,Fiction,Fiction
1,Fiction,Fiction
2,Fiction,Fiction
3,Fiction,Nonfiction
4,Fiction,Fiction
...,...,...
595,Nonfiction,Nonfiction
596,Nonfiction,Fiction
597,Nonfiction,Nonfiction
598,Nonfiction,Nonfiction


In [34]:
predictions_df["correct_prediction"] = np.where(predictions_df["actual_categories"] == predictions_df["predicted_categories"], 1, 0)

In [35]:
predictions_df["correct_prediction"].sum() / len(predictions_df)

0.7783333333333333

In [36]:
isbns = []
predicted_categories = []

missing_categories = books.loc[books["simple_categories"].isnull(), ["isbn13", "description"]].reset_index(drop=True)

In [37]:
for i in tqdm(range(0, len(missing_categories))):
    sequence = missing_categories["description"][i]
    predicted_categories += [generate_predictions(sequence, fiction_categories)]
    isbns += [missing_categories["isbn13"][i]]

100%|████████████████████████████████████████████████████████████████████████████| 1454/1454 [1:03:36<00:00,  2.62s/it]


In [38]:
missing_predicted_categories = pd.DataFrame({"isbn13": isbns, "predicted_categories": predicted_categories})

In [39]:
missing_predicted_categories

,isbn13,predicted_categories
0,9780002261982,Fiction
1,9780006280897,Nonfiction
2,9780006280934,Nonfiction
3,9780006380832,Nonfiction
4,9780006470229,Fiction
...,...,...
1449,9788125026600,Nonfiction
1450,9788171565641,Fiction
1451,9788172235222,Fiction
1452,9788173031014,Nonfiction


In [40]:
books = pd.merge(books, missing_predicted_categories, on="isbn13", how="left")
books["simple_categories"] = np.where(books["simple_categories"].isnull(), books["predicted_categories"], books["simple_categories"])
books = books.drop(columns = ["predicted_categories"])

In [41]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missin_description,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,0,Gilead,9780002005883: A NOVEL THAT READERS and critic...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,0,Spider's Web: A Novel,9780002261982: A new 'Christie for Christmas' ...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,0,Rage of angels,"9780006178736: A memorable, mesmerizing heroin...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,0,The Four Loves,9780006280897: Lewis' work on the nature of lo...,Nonfiction
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,0,The Problem of Pain,"9780006280934: ""In The Problem of Pain, C.S. L...",Nonfiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,0,Mistaken Identity,9788172235222: On A Train Journey Home To Nort...,Fiction
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,0,Journey to the East,9788173031014: This book tells the tale of a m...,Nonfiction
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623: Wisdom to Create a Life of Pass...,Fiction
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535: This collection of the timeless...,Nonfiction


In [42]:
books[books["categories"].str.lower().isin([
    "romance",
    "science fiction",
    "scifi",
    "fantasy",
    "horror",
    "mystery",
    "thriller",
    "comedy",
    "crime",
    "historical",
])]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missin_description,title_and_subtitle,tagged_description,simple_categories
24,9780006513087,0006513085,Gravity,Tess Gerritsen,Science fiction,http://books.google.com/books/content?id=KI66c...,Emma Watson a research physician has been trai...,2004.0,4.04,342.0,8024.0,0,Gravity,9780006513087: Emma Watson a research physicia...,Nonfiction
475,9780099410355,0099410354,Traitor,Matthew Woodring Stover,Science fiction,http://books.google.com/books/content?id=VbICO...,"From the depths of catastrophe, a glimmer of h...",2002.0,4.00,320.0,6765.0,0,Traitor,"9780099410355: From the depths of catastrophe,...",Fiction
478,9780099422341,0099422344,Yeats is Dead!,Joseph O'Connor,Comedy,http://books.google.com/books/content?id=DrE3I...,"In aid of Amnesty International, this is a bri...",2002.0,3.39,298.0,34.0,0,Yeats is Dead!: A Novel by Fifteen Irish Writers,9780099422341: In aid of Amnesty International...,Fiction
491,9780099446729,0099446723,Blackwood Farm,Anne Rice,Horror,http://books.google.com/books/content?id=cIn8T...,"Lestat Is Back, Saviour And Demon, Presiding O...",2003.0,3.86,774.0,26145.0,0,Blackwood Farm,"9780099446729: Lestat Is Back, Saviour And Dem...",Fiction
1090,9780261102422,0261102427,The Silmarillion,John Ronald Reuel Tolkien,Fantasy,http://books.google.com/books/content?id=22ePu...,Tolkien's Silmarillion is the core work of the...,1999.0,3.91,384.0,253.0,0,The Silmarillion,9780261102422: Tolkien's Silmarillion is the c...,Fiction
1435,9780340837955,0340837950,Stranger in a Strange Land,Robert A. Heinlein,Science fiction,http://books.google.com/books/content?id=ZQhiP...,"Epic, entertaining, Stranger in a Strange Land...",2005.0,3.92,672.0,563.0,0,Stranger in a Strange Land,"9780340837955: Epic, entertaining, Stranger in...",Fiction
1439,9780345251220,0345251229,Visions from Nowhere,William Arrow,Science fiction,NaN,"The first novel in the series, ""Return to the ...",1976.0,3.23,183.0,10.0,0,Visions from Nowhere,"9780345251220: The first novel in the series, ...",Fiction
2845,9780575075597,0575075597,Replay,Ken Grimwood,Fantasy,http://books.google.com/books/content?id=9vmNP...,At forty-three Jeff Winston is tired of his lo...,2005.0,4.16,272.0,412.0,0,Replay,9780575075597: At forty-three Jeff Winston is ...,Fiction
2860,9780590254762,0590254766,"The lion, the witch and the wardrobe",Clive Staples Lewis,Fantasy,NaN,Four English school children enter the magic l...,1995.0,4.21,189.0,860.0,0,"The lion, the witch and the wardrobe",9780590254762: Four English school children en...,Nonfiction
3288,9780739423851,0739423851,Wizard's Castle,Diana Wynne Jones,Fantasy,http://books.google.com/books/content?id=hB7hA...,Howl's moving castle - Eldest of three sisters...,2002.0,4.44,376.0,439.0,0,Wizard's Castle,9780739423851: Howl's moving castle - Eldest o...,Fiction


In [43]:
books.to_csv("books_with_categories.csv", index=False)